# SDialog dependencies

In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython


if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing Ollama (if you are not planning to use Ollama, you can just comment these lines to speed up the installation)
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/qanastek/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

Running in Jupyter Notebook


## Locally

Run following commands and then `Restart` your environment.

In [2]:
%%script false --no-raise-error
%pip install -e ..
%pip show sdialog

In order to run the next steps in a fast manner, we will start from an existing dialog generated using previous tutorials. If you haven't download yet `dialog_demo.json` locally, please download it from our GitHub repository using the following command:

In [3]:
# If dialog_demo.json is not present, download it
if os.path.exists("dialog_demo.json"):
    print("dialog_demo.json already exists")
else:
    !wget https://raw.githubusercontent.com/qanastek/sdialog/refs/heads/main/misc/audio/dialog_demo.json

dialog_demo.json already exists


In [4]:
import os
import json
from tqdm import tqdm

In [5]:
import sdialog
from sdialog import Dialog

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dialog

In [6]:
original_dialog = Dialog.from_file("dialog_demo.json")
original_dialog.print()

[dialog_id] 1752861530588
[model] client=<botocore.client.BedrockRuntime object at 0x7fc319ec91c0> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' region_name='us-east-1' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')
[seed] 226296126
--- Dialogue Begins ---
[DOCTOR] Hello there. I'm Dr. Smith. Welcome to my office. What brings you in to see me today?
[PATIENT] Good morning, Dr. Smith. I'm John Doe. I've been dealing with a persistent cough and feeling pretty tired for about three weeks now. It's starting to wear me down, and I thought I should get it checked out.
[DOCTOR] I'm sorry to hear you've been feeling unwell, Mr. Doe. A persistent cough and fatigue can certainly be troublesome. Let's get some more details. Can you describe your cough? Is it dry or productive?
[PATIENT] Well, Dr. Smith, it's mostly a dry cough. It's particularly bothersome at night when I'm trying to sleep. You know, it reminds me of the time I was teaching about the Civil War and

# Audio Generation

### Instanciate voices database

In [7]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
dummy_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-kokoro")

[2025-10-08 15:47:26] INFO:root:Voice database populated with 300 voices


Now we are trying to fetch a voice for a `20` years old `male`:

In [8]:
dummy_voice_database.get_voice(genre="male", age=20)

{'identifier': 'am_adam', 'voice': 'am_adam'}

### Instanciate TTS model

In [9]:
!pip install -q kokoro>=0.9.4 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

32512

In [10]:
from sdialog.audio.tts_engine import KokoroTTS
tts_engine = KokoroTTS()

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## Setup stage: Audio Dialog and Audio Pipeline

In [11]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [12]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

Identify speakers names:

In [13]:
print("Speaker 1:", dialog.speakers_names["speaker_1"])
print("Speaker 2:", dialog.speakers_names["speaker_2"])

Speaker 1: DOCTOR
Speaker 2: PATIENT


## Step 1 : Concatenated utterances

Instanciate the audio pipeline in order to use `Kokoro` (`tts_engine`) as the TTS model and save the outputs of all the dialogs into the directory `./audio_outputs`.

The voices are sampled from the `dummy_voice_database` based on the persona attributes `age` and `gender`, as assigned during the original textual dialog.

In [14]:
os.makedirs("./audio_outputs", exist_ok=True)
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_engine,
    dir_audio="./audio_outputs",
)
# audio_pipeline = AudioPipeline() # Can also be used with default values

Perform the inference of the audio pipeline on the previously converted dialog. In this case we will focus on generating the "unprocessed" audio, which consist of the agregation of all utterances from the dialog. Rather than using the dialog identifier as the name of the directory, we are using here a custom directory name `demo_dialog_kokoro` which will be saved at `./audio_outputs/demo_dialog_kokoro/`. 

In [15]:
# Generate the audio for the dialog
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    do_step_1=True,
    do_step_2=False,
    do_step_3=False,
    dialog_dir_name="demo_dialog_kokoro",
)

# Path to the audio of the first stage of the audio pipeline
print("Audio generated successfully at:", dialog.audio_step_1_filepath)

[2025-10-08 15:47:31] INFO:root:Dialog audio dir path: ./audio_outputs
[2025-10-08 15:47:31] INFO:root:Audio dialog loaded from the existing file (1752861530588) successfully!
/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/pydantic/main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `list[str]` - serialized value may not be as expected [input_value={}, input_type=dict])
  return self.__pydantic_serializer__.to_json(
[2025-10-08 15:47:31] INFO:root:Audio dialog saved to the existing file (1752861530588) successfully at the end of the pipeline!


Audio generated successfully at: ./audio_outputs/demo_dialog_kokoro/exported_audios/audio_pipeline_step1.wav


In [16]:
from IPython.display import Audio, display

display(Audio(dialog.audio_step_1_filepath, autoplay=True))